# Projet sante publique france

## Exploratoration initiale

Les données sont accessibles sous forme de `CSV` (**Le fichier CSV est assez conséquent env. 2GB.**) ou éventuellement si la base devient trop lourde pour être chargée en RAM d'un bloc, les données peuvent être restaurées dans une base de données MongoDB. 

Pour charger les données, un script est disponible dans le répertoire /src. Le plus simple est de créer un environement virtuel, installer les dépendances et exécuter `make data`.

## Nettoyage des données 

Les données étant crées par les utilisateurs, les données sont "sales". Il faut retirer toutes les données qui sont des erreurs de saisie.
   * Données provenant de sources fiables vs. sources non fiables.
       * Certaines données sont entrées par des organismes "fiables" exemple carrefour, fleury-michon...
   * Retirer les produits n'ayant pas de nom (product_name == nan).
   * Sauvegarde des données dans un fichier intermédiaire


Le nettoyage des données sera traité plus en détails dans le notebook suivant. Les données doivent être nettoyées de manière statistique.

### Chargement des données

NB: Les données sont assez lourdes, le chargement peut prendre un certain temps


In [1]:
from IPython.display import display
import numpy as np
import pandas as pd

pd.options.display.max_columns = 10
pd.options.display.max_colwidth = 20

In [2]:
dtype = {'code' : str,
         'emb_codes' : str,
         'emb_codes_tags' : str,
         'first_packaging_code_geo' : str,
         'cities_tags' : str,
         'additives' : str,
         'ingredients_from_palm_oil_tags' : str,
        }

In [ ]:
%%time
data = pd.read_csv('../data/raw/products.csv', 
                   delimiter='\t', 
                   low_memory=True, 
                   dtype=dtype)

In [7]:
with open('../data/raw/products.csv', 'r') as file:
    for i, row in enumerate(file.readlines()):
        if i == 0:
            break

In [8]:
row.split('\t')

['code',
 'url',
 'creator',
 'created_t',
 'created_datetime',
 'last_modified_t',
 'last_modified_datetime',
 'product_name',
 'generic_name',
 'quantity',
 'packaging',
 'packaging_tags',
 'brands',
 'brands_tags',
 'categories',
 'categories_tags',
 'categories_en',
 'origins',
 'origins_tags',
 'manufacturing_places',
 'manufacturing_places_tags',
 'labels',
 'labels_tags',
 'labels_en',
 'emb_codes',
 'emb_codes_tags',
 'first_packaging_code_geo',
 'cities',
 'cities_tags',
 'purchase_places',
 'stores',
 'countries',
 'countries_tags',
 'countries_en',
 'ingredients_text',
 'allergens',
 'allergens_en',
 'traces',
 'traces_tags',
 'traces_en',
 'serving_size',
 'serving_quantity',
 'no_nutriments',
 'additives_n',
 'additives',
 'additives_tags',
 'additives_en',
 'ingredients_from_palm_oil_n',
 'ingredients_from_palm_oil',
 'ingredients_from_palm_oil_tags',
 'ingredients_that_may_be_from_palm_oil_n',
 'ingredients_that_may_be_from_palm_oil',
 'ingredients_that_may_be_from_palm_

On ne récupère que les données concernant la France et les pays voisins francophones.

In [ ]:
data = data[(data['countries_tags'] == 'en:france') | 
            (data['countries_tags'] == 'en:belgium') |
            (data['countries_tags'] == 'en:switzerland')|
            (data['countries_tags'] == 'en:luxembourg')]

On drop les timestamps (inutiles puisqu'on a les datetimes).

On convertit alors les timestamp au format pd.Datetime et les champs finissant par `_n` sont des entiers.

NB: Le type `int` ne peut pas contenir de nan, donc on remplace ces derniers par -999

In [ ]:
for col in data.columns:
    if col.endswith('_t'):
        data.drop(col, axis=1, inplace=True)
    elif col.endswith('_datetime'):
        data[col] = pd.to_datetime(data[col])
        data[col] = data[col].map(lambda x : x.replace(tzinfo=None))
    elif col.endswith('_n'):
        print(col)
        # Replace nan by int to allow conversion of dtype from float to int
        data[col].fillna(-999, inplace=True)
        # 8 bits should be enough be use 16 to be sure
        data[col] = data[col].astype('int16')

On peut regarder les contributeurs ainsi que le nombre de contributions qu'ils ont fourni

In [ ]:
with pd.option_context("display.max_rows", 30):
    display(data['creator'].value_counts().head(30))

On peut commencer par définir des sources "fiables". En effet, certains producteurs renseignent eux-même les données (les données sont de bonnes qualité)

In [ ]:
TRUSTED_SOURCES = ['casino', 'carrefour', 'ldc', 'fleury-michon', 'ferrero', 'systeme-u', 'barilla']

On drop les produits pour lesquels on a pas de nom.

In [ ]:
print(f'Nombre de lignes avant supression {data.shape[0]}')
data.drop(data[data['product_name'].isna()].index, axis=0, inplace=True)
print(f'Nombre de lignes après supression {data.shape[0]}')

In [ ]:
data['creator'].value_counts()

In [ ]:
data['trusted_src'] = data['creator'].map(lambda x : True if x in TRUSTED_SOURCES else False)

In [ ]:
data['main_category_fr']

On supprime les colonnes vides

In [ ]:
col1 = list(data.columns.values)
data.dropna(axis=1, inplace=True, how='all')
col2 = list(data.columns.values)

print(f"Colonnes supprimées : {list(set(col1) - set(col2))}")

Sauvegarde des données partiellement nettoyées

In [ ]:
# to_pickle run faster than to_csv
data.to_pickle('../data/interim/products_interim.pickle')